<a href="https://colab.research.google.com/github/Capstone-Team-CH2-PS379/Machine-Learning/blob/main/Speech_Recognition_Model_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load Data

In [ ]:
!pip install kaggle

In [ ]:
import os

# Set Kaggle API Key as environment variable
os.environ['KAGGLE_USERNAME'] = "your_kaggle_username"
os.environ['KAGGLE_KEY'] = "your_kaggle_api_key"

In [ ]:
# Download dataset
!kaggle datasets download -d mozillaorg/common-voice

common-voice.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# Extract zipfile to folder
import zipfile

with zipfile.ZipFile('common-voice.zip', 'r') as zip_ref:
    zip_ref.extractall('common-voice')

In [ ]:
print(os.listdir('/content/common-voice'))

['cv-valid-dev.csv', 'cv-valid-train.csv', 'cv-other-test', 'cv-invalid', 'cv-other-train.csv', 'cv-invalid.csv', 'cv-other-train', 'cv-valid-test.csv', 'LICENSE.txt', 'cv-valid-train', 'cv-valid-test', 'cv-other-dev.csv', 'cv-other-dev', 'cv-other-test.csv', 'README.txt', 'cv-valid-dev']


In [ ]:
import pandas as pd

# Read the CSV file
file_path = '/content/common-voice/cv-valid-train.csv'
data_train = pd.read_csv(file_path)
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195776 entries, 0 to 195775
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   filename    195776 non-null  object 
 1   text        195776 non-null  object 
 2   up_votes    195776 non-null  int64  
 3   down_votes  195776 non-null  int64  
 4   age         73768 non-null   object 
 5   gender      74059 non-null   object 
 6   accent      64711 non-null   object 
 7   duration    0 non-null       float64
dtypes: float64(1), int64(2), object(5)
memory usage: 11.9+ MB


In [ ]:
# Set the option to display maximum column width
pd.set_option('display.max_colwidth', None)
data_train.head()

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-train/sample-000000.mp3,learn to recognize omens and follow them the old king had said,1,0,NaN,NaN,NaN,NaN
1,cv-valid-train/sample-000001.mp3,everything in the universe evolved he said,1,0,NaN,NaN,NaN,NaN
2,cv-valid-train/sample-000002.mp3,you came so that you could learn about your dreams said the old woman,1,0,NaN,NaN,NaN,NaN
3,cv-valid-train/sample-000003.mp3,so now i fear nothing because it was those omens that brought you to me,1,0,NaN,NaN,NaN,NaN
4,cv-valid-train/sample-000004.mp3,if you start your emails with greetings let me be the first to welcome you to earth,3,2,NaN,NaN,NaN,NaN


# Pre-processing Data

In [ ]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

In [ ]:
print(tf.__version__)

2.13.1


In [ ]:
!pip install tensorflow-io[tensorflow]
import tensorflow_io as tfio

In [ ]:
# Fungsi untuk membaca file audio dan mengorganisasikannya ke dalam tensor
def read_and_convert_to_tensor(csv_path, audio_folder):
    # Baca file CSV
    df = pd.read_csv(csv_path)

    data = []
    labels = []

    for index, row in df.iterrows():
        file_name = row['filename']  # Ganti dengan nama kolom yang sesuai
        label = row['text']
        file_path = os.path.join(audio_folder, file_name)

        # Pastikan file audio ada
        if os.path.exists(file_path):
            # Baca file audio menggunakan TensorFlow
            audio_binary = tf.io.read_file(file_path)

            # Dekode audio sesuai format MP3
            audio = tfio.audio.decode_mp3(audio_binary)

            # Sesuaikan bentuk tensor jika perlu
            # Misalnya, jika model Anda mengharapkan dimensi yang berbeda
            # audio = tf.reshape(audio, (new_shape))

            data.append(audio.numpy())
            labels.append(label)
        else:
            print(f"File not found: {file_path}")

    return np.array(data), np.array(labels)

# Menggunakan kode yang telah Anda berikan
csv_path = '/content/common-voice/cv-valid-train.csv'
audio_folder = '/content/common-voice/cv-valid-train'
train_data, train_labels = read_and_convert_to_tensor(csv_path, audio_folder)

# Cetak bentuk dari data dan label untuk memastikan semuanya sesuai
print("Train Data Shape:", train_data.shape)
print("Train Labels Shape:", train_labels.shape)